# Dollar Cost Averaging Strategy Explained

## Introduction

Dollar Cost Averaging (DCA) is an investment strategy where an investor divides the total amount to be invested into periodic purchases of a target asset, 
aiming to reduce the impact of volatility. This approach can be particularly effective in volatile markets, as it spreads out the cost basis over time.

In this notebook, we will use **backtrader**, a Python library for backtesting trading strategies, to assess how the DCA strategy would have performed historically on the SPY ETF, 
an exchange-traded fund tracking the S&P 500 index.

### Objectives:
1. Fetch historical data for SPY ETF using `yfinance`.
2. Implement the DCA strategy in **backtrader**.
3. Visualize and interpret the results.

### Libraries Used:
- `yfinance`: Fetch financial data from Yahoo Finance.
- `backtrader`: Backtesting library for financial strategies.
- `matplotlib`: Plotting and visualization.
- `numpy` and `pandas`: Data manipulation.

In [1]:
import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import backtrader as bt

## Data Retrieval

We fetch historical price data for the SPY ETF using `yfinance`. The data spans approximately 10 years (2520 trading days). 
This data will be used as input for the backtesting process.

### Steps:
1. Define the target stock (SPY) and the date range for analysis.
2. Use `yfinance.download` to fetch adjusted closing prices.
3. Prepare the data for compatibility with `backtrader`.

Below is the code to fetch and preprocess the data.


In [2]:
# Function to fetch stock data using yfinance
def get_data(stocks, start, end):
    """
    Fetch historical price data for specified stocks.

    Args:
        stocks (list): List of stock tickers.
        start (datetime): Start date for data fetching.
        end (datetime): End date for data fetching.

    Returns:
        pd.DataFrame: DataFrame containing stock prices.
    """
    stockData = yf.download(stocks, start, end)
    return stockData

# Define stock ticker and date range
stockList = ['SPY']  # SPY ETF tracks S&P 500
endDate = datetime.datetime.now()
startDate = endDate - datetime.timedelta(days=2520)  # Approx. 10 years of data

# Fetch data
stockData = get_data(stockList, startDate, endDate)

# Preprocess data for backtrader compatibility
stockData.columns = stockData.columns.droplevel(1)  # Flatten MultiIndex
stockData.columns.name = None

# Verify the actual data start date
actualStart = stockData.index[0]

data = bt.feeds.PandasData(dataname=stockData)

[*********************100%***********************]  1 of 1 completed


## Strategy Implementation

We implement two strategies:
1. **Buy and Hold:** Invest the entire available cash at the start and hold until the end of the investment period.
2. **Buy and Hold with Additional Investments:** Start with a fixed amount and add a specified amount of cash monthly, simulating a DCA approach.

Each strategy calculates key metrics such as return on investment (ROI), gross returns, and annualized performance.


In [3]:
class BuyAndHold(bt.Strategy):
    def start(self):
        # Initial cash at the start of the strategy
        self.val_start = self.broker.get_cash()

    def nextstart(self):
        # Invest all available cash (minus a safety buffer) in the stock
        size = math.floor((self.broker.get_cash() - 10) / self.data[0])
        self.buy(size=size)

    def stop(self):
        # Calculate and display the performance metrics
        self.roi = (self.broker.get_value() / self.val_start) - 1
        print('-' * 50)
        print('BUY & HOLD')
        print(f'Starting Value:  ${self.val_start:,.2f}')
        print(f'ROI:              {self.roi * 100.0:.2f}%')
        print(f'Annualised:       {100 * ((1 + self.roi) ** (365 / (endDate - actualStart).days) - 1):.2f}%')
        print(f'Gross Return:    ${self.broker.get_value() - self.val_start:,.2f}')

### Buy and Hold with Monthly Investments (DCA)

This strategy simulates a Dollar Cost Averaging (DCA) approach. It:
1. Starts with a small amount of initial capital.
2. Adds a fixed monthly amount at random intervals within a specified range.
3. Invests all available cash each month.

The strategy evaluates metrics such as:
- Total cost of investments.
- Gross and percentage returns.
- ROI and fund value.


In [4]:
class BuyAndHold_More_Fund(bt.Strategy):
    params = dict(
        monthly_cash=1000,  # Amount to add monthly
        monthly_range=[5, 20]  # Random days of the month for investment
    )

    def __init__(self):
        # Initialize strategy variables
        self.order = None
        self.totalcost = 0  # Track total invested amount including commissions
        self.cost_wo_bro = 0  # Total cost excluding commissions
        self.units = 0  # Number of units bought
        self.times = 0  # Number of times investments were made

    def log(self, txt, dt=None):
        # Logging function for tracking activity
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def start(self):
        # Initialize broker settings
        self.broker.set_fundmode(fundmode=True, fundstartval=100.0)
        self.cash_start = self.broker.get_cash()
        self.val_start = 100.0

        # Add a timer for monthly investments
        self.add_timer(
            when=bt.timer.SESSION_START,
            monthdays=[i for i in self.p.monthly_range],
            monthcarry=True
        )

    def notify_timer(self, timer, when, *args):
        # Add monthly cash and invest
        self.broker.add_cash(self.p.monthly_cash)
        target_value = self.broker.get_value() + self.p.monthly_cash - 10
        self.order_target_value(target=target_value)

    def notify_order(self, order):
        # Track order completion and log execution details
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'BUY EXECUTED, Price {order.executed.price:.2f}, Cost {order.executed.value:.2f}, '
                    f'Comm {order.executed.comm:.2f}, Size {order.executed.size:.0f}'
                )
                self.units += order.executed.size
                self.totalcost += order.executed.value + order.executed.comm
                self.cost_wo_bro += order.executed.value
                self.times += 1

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def stop(self):
        # Calculate and display the performance metrics
        self.roi = (self.broker.get_value() / self.cash_start) - 1
        self.froi = (self.broker.get_fundvalue() - self.val_start)
        value = self.datas[0].close * self.units + self.broker.get_cash()
        print('-' * 50)
        print('BUY & BUY MORE')
        print(f'Time in Market: {(endDate - actualStart).days / 365:.1f} years')
        print(f'#Times:         {self.times:.0f}')
        print(f'Value:         ${value:,.2f}')
        print(f'Cost:          ${self.totalcost:,.2f}')
        print(f'Gross Return:  ${value - self.totalcost:,.2f}')
        print(f'Gross %:        {(value / self.totalcost - 1) * 100:.2f}%')
        print(f'ROI:            {self.roi * 100.0:.2f}%')
        print(f'Fund Value:     {self.froi:.2f}%')
        print(f'Annualised:     {100 * ((1 + self.froi / 100) ** (365 / (endDate - actualStart).days) - 1):.2f}%')
        print('-' * 50)


### Custom Commission Scheme

A fixed commission scheme is implemented to simulate realistic trading costs. 
This ensures that the backtesting results account for brokerage fees.

#### Key Parameters:
- `commission`: The fixed amount charged per transaction.
- `stocklike`: Indicates that the asset behaves like a stock.
- `commtype`: Specifies the type of commission (fixed in this case).

In [5]:
class FixedCommisionScheme(bt.CommInfoBase):
    params = (
        ('commission', 10),  # Fixed commission per trade
        ('stocklike', True),  # Treat the asset like a stock
        ('commtype', bt.CommInfoBase.COMM_FIXED)  # Fixed commission type
    )

    def _getcommission(self, size, price, pseudoexec):
        # Calculate commission based on fixed amount
        return self.p.commission

### Running the Strategies

The `run` function executes the two strategies (`BuyAndHold` and `BuyAndHold_More_Fund`) with the following steps:
1. **Initialize `Cerebro` engine:** The main framework for running backtests in `backtrader`.
2. **Add data feed:** Input the stock data fetched earlier.
3. **Add strategies:** Include the two strategies for comparison.
4. **Set broker settings:** Define initial cash, commission schemes, and broker arguments.
5. **Execute and visualize results:** Run the strategies and generate performance plots.

In [6]:
def run(data):
    # BUY and HOLD
    cerebro = bt.Cerebro()
    cerebro.adddata(data)  # Add data feed to Cerebro
    cerebro.addstrategy(BuyAndHold)  # Add Buy and Hold strategy

    # Configure broker for the Buy and Hold strategy
    broker_args = dict(coc=True)  # Enable cash-on-cash calculation
    cerebro.broker = bt.brokers.BackBroker(**broker_args)
    comminfo = FixedCommisionScheme()  # Use the fixed commission scheme
    cerebro.broker.addcommissioninfo(comminfo)
    cerebro.broker.set_cash(100000)  # Set initial cash

    # BUY and BUY MORE
    cerebro1 = bt.Cerebro()
    cerebro1.adddata(data)  # Add data feed to Cerebro
    cerebro1.addstrategy(BuyAndHold_More_Fund)  # Add DCA strategy

    # Configure broker for the DCA strategy
    cerebro1.broker = bt.brokers.BackBroker(**broker_args)
    cerebro1.broker.addcommissioninfo(comminfo)
    cerebro1.broker.set_cash(1000)  # Start with a smaller cash amount

    # Run strategies
    cerebro1.run()
    cerebro.run()

    # Plot results
    cerebro.plot(iplot=False, style='candlestick')  # Visualize Buy and Hold results
    cerebro1.plot(iplot=False, style='candlestick')  # Visualize DCA strategy results

# Run the backtesting function with the prepared data
run(data)

2018-01-08, BUY EXECUTED, Price 273.42, Cost 1913.94, Comm 10.00, Size 7
2018-01-23, BUY EXECUTED, Price 282.69, Cost 848.07, Comm 10.00, Size 3
2018-02-06, BUY EXECUTED, Price 263.93, Cost 1055.72, Comm 10.00, Size 4
2018-02-21, BUY EXECUTED, Price 271.40, Cost 1085.60, Comm 10.00, Size 4
2018-03-06, BUY EXECUTED, Price 272.19, Cost 816.57, Comm 10.00, Size 3
2018-03-21, BUY EXECUTED, Price 270.95, Cost 1083.80, Comm 10.00, Size 4
2018-04-06, BUY EXECUTED, Price 265.64, Cost 1062.56, Comm 10.00, Size 4
2018-04-23, BUY EXECUTED, Price 266.61, Cost 799.83, Comm 10.00, Size 3
2018-05-08, BUY EXECUTED, Price 266.92, Cost 1067.68, Comm 10.00, Size 4
2018-06-06, BUY EXECUTED, Price 275.10, Cost 1100.40, Comm 10.00, Size 4
2018-06-21, BUY EXECUTED, Price 275.97, Cost 827.91, Comm 10.00, Size 3
2018-07-06, BUY EXECUTED, Price 273.11, Cost 1092.44, Comm 10.00, Size 4
2018-07-23, BUY EXECUTED, Price 279.68, Cost 839.04, Comm 10.00, Size 3
2018-08-07, BUY EXECUTED, Price 284.64, Cost 1138.56, Co

### Buy And Hold
![Chart](/home/usamabuttar/Projects/QuantPy/Backtest/BuyAndHold.png "Buy And Hold")

### Dollar Cost Average
![Chart](/home/usamabuttar/Projects/QuantPy/Backtest/DollarCostAverage.png "Dollar Cost Average")

## Final Thoughts

The Dollar Cost Averaging (DCA) strategy offers a disciplined approach to investing, particularly in volatile markets. 
By comparing the Buy and Hold strategy with the DCA strategy, we can evaluate the benefits of systematic investing.

### Next Steps:
1. Experiment with different monthly cash amounts and investment intervals.
2. Test the strategies on other stocks or ETFs.
3. Integrate risk management techniques to enhance the strategies.